In [25]:
import requests
from io import StringIO
import pandas as pd
import numpy as np
import datetime
import os
import csv

def crawlPrice(date):  #input 是日期
    r = requests.post('http://app.twse.com.tw/ch/trading/exchange/MI_INDEX/MI_INDEX.php', data={
        'download': 'csv',
        'qdate':date,
        'selectType':'ALL',
    })
    r.encoding = 'big5'
    df = pd.read_csv(StringIO("\n".join([i.translate({ord(c): None for c in ' '}) 
                                         for i in r.text.split('\n') 
                                         if len(i.split('",')) == 16 and i[0] != '='])), header=0)
    df.set_index('證券代號', inplace=True)
    df.columns = ['證券名稱', '成交量', '成交筆數', '成交金額', '開盤價', '最高價', '最低價', '收盤價',
                  '漲跌(+/-)', '漲跌價差', '最後揭示買價', '最後揭示買量', '最後揭示賣價', '最後揭示賣量', '本益比']
    
    df['成交量'] /= 1000
    df = df.drop(['漲跌(+/-)','證券名稱','最後揭示買量','最後揭示賣量'], axis=1)
    df = df.replace('--', np.nan)
    df = df.apply(pd.to_numeric)
    df = df.apply(pd.to_numeric)
    assert len(set(df.index)) == len(df.index)
    
    return df #output 是個股當日價格 dataframe

# 爬股價

data = {}
# time = datetime.datetime.now() #爬最新

date = open('date_downloaded', 'r')
date_str = date.read()
time = datetime.datetime.strptime(date_str,'%Y%m%d') #爬歷史

while international_time_str != '20171001': #爬到哪一天停止
    
    # 假如日月 < 9 要補零
    month_str = str(time.month) if time.month > 9 else '0' + str(time.month)
    day_str = str(time.day) if time.day > 9 else '0' + str(time.day)
    
    # e.x 20100101
    taiwan_time_str = str(time.year - 1911) + '/' + month_str + '/' + day_str
    international_time_str = str(time.year) + month_str + day_str
    
    if not os.path.exists('stockprice/{}.csv'.format(international_time_str)):
    
        print('parsing', international_time_str)
        # 使用 crawPrice 爬資料
    
        try:
            data = crawlPrice(taiwan_time_str)
            df = pd.DataFrame(data)
            df.to_csv('stockprice/{}.csv'.format(international_time_str))            
            print('success!')
        
        except:
            # 假日爬不到
            print('fail! check the date is holiday')
    
    '''
    else:  # 顯示當日已存
        print('Already have {}'.format(international_time_str))
    '''
    
    # 減一天
    time -= datetime.timedelta(days=1)
    
    # 紀錄已爬到哪一天
    f = open('date_downloaded', 'w')
    f.write(international_time_str)
    f.close()
    
print("done")

parsing 20171010
fail! check the date is holiday
parsing 20171009
fail! check the date is holiday
parsing 20171008
fail! check the date is holiday
parsing 20171007
fail! check the date is holiday
parsing 20171006
success!
parsing 20171005
success!
parsing 20171004
fail! check the date is holiday
parsing 20171003
success!
parsing 20171002
success!
parsing 20171001
fail! check the date is holiday
done
